In [ ]:
import torch
import numpy as np
import keras
from keras import backend as K

In [ ]:
P = np.load('P.npy')

In [ ]:
# P is the joint probabilities for this batch (Keras loss functions call this y_true)
# activations is the low-dimensional output (Keras loss functions call this y_pred)
def tsne2(P, activations):
#     d = K.shape(activations)[1]
    d = 2 # TODO: should set this automatically, but the above is very slow for some reason
    n = 5000 # TODO: should set this automatically
    v = d - 1.
    eps = K.variable(10e-15) # needs to be at least 10e-8 to get anything after Q /= K.sum(Q)
    sum_act = K.sum(K.square(activations), axis=1)
    Q = K.reshape(sum_act, [-1, 1]) + -2 * K.dot(activations, K.transpose(activations))
    Q = (sum_act + Q) / v
    Q = K.pow(1 + Q, -(v + 1) / 2)
    Q *= K.variable(1 - np.eye(n))
    Q /= K.sum(Q)
    Q = K.maximum(Q, eps)
    C = K.log((P + eps) / (Q + eps))
    C = K.sum(P * C)
    return C

In [ ]:
P_reshaped = P.reshape(60000, 5000)

In [ ]:
np.all(np.equal(P_reshaped[:5000, :5000], P_reshaped[:5000, :5000].T))

In [ ]:
def tsne(P, activations):
    n, d = activations.shape
    alpha = d - 1
    eps = 10e-8
    act_pdist = torch.cdist(x1=activations, x2=activations, p=2.)
    terms = (1 + act_pdist ** 2 / alpha) ** (-(alpha + 1) / 2)
    idx = torch.arange(n)
    terms[idx, idx] = 0
    Q = terms / terms.sum(dim=1, keepdims=True)
    Q = torch.clamp(Q, min=eps)
    C = P * torch.log((P + eps) / (Q + eps))
    C[idx, idx] = 0  # remove i == j terms
    return C.sum()

In [ ]:
activation = np.random.rand(5000, 2)

In [ ]:
C1 = tsne(torch.from_numpy(P), torch.from_numpy(activation))

In [ ]:
batch_size = 5000
C1 = tsne2(K.variable(P), K.variable(activation))

In [ ]:
C1

In [ ]:
import torch
from tqdm.auto import tqdm

In [ ]:

def nearest_neighbors(x, top_k, device):
    """
    calculate the nearest neighbors of x, return the
    :param x: for matrix to calculate nearest neighbor
    :param top_k: number of the nearest neighbor to be returned
    :param device: device used during computation
    :return:
            ground_min_dist_square: torch.tensor (n, ) distance to the nearest neighbor
            topk_neighbors: torch.tensor (n, top_k) the index of the top-k nearest neighbors;
    """
    batch_size = 2001
    x = x.to(device)
#     if x.shape[0] * x.shape[1] < batch_size * 200:  # direct computes the whole matrix
#         dist = torch.cdist(x1=x, x2=x, p=2)  # (n, n)
#         sorted_dist, indices = torch.sort(dist, dim=1, descending=False)
#         ground_min_dist_square = sorted_dist[:, 1]  # the 0-th column is the distance to oneself
#         topk_neighbors = indices[:, 1:1 + top_k]
#         topk_dists = sorted_dist[:, 1:1 + top_k]
#     else:  # calculate the nearest neighbors in batches
    num_iter = x.shape[0] // batch_size + 1
    topk_neighbors_list = list()
    ground_min_dist_square_list = list()
    sorted_dist_list = list()
    for i in tqdm(torch.arange(num_iter), desc='computing nearest neighbors'):
        print(i.item())
        batch_x = x[i * batch_size: (i + 1) * batch_size, :]
        dist = torch.cdist(x1=batch_x, x2=x, p=2)  # (n, n)
        print(dist.shape)
        sorted_dist, indices = torch.sort(dist, dim=1, descending=False)
        batch_ground_min_dist_square = sorted_dist[:, 1]  # the 0-th column is the distance to oneself
        batch_topk_neighbors = indices[:, 1:1 + top_k]
        topk_neighbors_list.append(batch_topk_neighbors.cpu())
        ground_min_dist_square_list.append(batch_ground_min_dist_square.cpu())
        sorted_dist_list.append(sorted_dist[:, 1:1 + top_k].cpu())
    ground_min_dist_square = torch.cat(ground_min_dist_square_list, dim=0)
    topk_neighbors = torch.cat(topk_neighbors_list, dim=0)
    topk_dists = torch.cat(sorted_dist_list, dim=0)
    return ground_min_dist_square.cpu(), topk_neighbors.cpu(), topk_dists.cpu()


In [ ]:
X_train_transformed_tensor = torch.load('X_train_transformed_tensor.torch')
X_train = torch.load('X_train.torch')

In [ ]:
X_transfomed_nn = nearest_neighbors(X_train_transformed_tensor, 10, 'cuda:0')

In [ ]:
X_nn = nearest_neighbors(torch.from_numpy(X_train), 10, 'cuda:1')

In [ ]:
a = X_nn[1][:, 0]

In [ ]:
b = X_transfomed_nn[1][:, 0]

In [ ]:
(a == b).sum() * 1. / len(a)

In [ ]:
X_train_transformed_tensor[0]

In [ ]:
y = X_train_transformed_tensor[0]
best_i = -1
best_d = 1e8
for i, x in enumerate(X_train_transformed_tensor[0:]):
    d = ((x - y) ** 2).sum()
    if d < best_d and d > 0:
        best_d = d
        best_i = i

In [ ]:
best_i

In [ ]:
a

In [ ]:
b

In [ ]:
len(a==b)

In [ ]:
(a==b).sum()